## Question 1

In [14]:
p = power(10,-3)
1-power(1 - p,8)

7972055930055972007999/1000000000000000000000000

In [12]:
round(Out[3],8)

0.00797206

## Question 2

In [4]:
q = 256
n = 255
p = power(10,-3)

In [5]:
def P_fail(p,n,t):
    sigma = 0
    
    for j in range(t+1):
        sigma += binomial(n,j)*power(p,j)*power(1-p,n-j)
        
    return 1-sigma

In [6]:
# largest k for P_fail < 0.02
t = 1
while P_fail(p,n,t) > 2/100:
    t += 1
k = n-2*t
print(k)

251


In [7]:
# largest k for P_fail < 0.001
t = 1
while P_fail(p,n,t) > 1/1000:
    t += 1
k = n-2*t
print(k)

249


In [8]:
# largest k for P_fail < 10^-10
t = 1
while P_fail(p,n,t) > power(10,-10):
    t += 1
k = n-2*t
print(k)

239


## Question 3

In [1]:
q = 256
n = 255
p = power(10,-3)
F.<g> = FiniteField(q)
Fpol.<X> = PolynomialRing(F,'X')

# choose evaluation points :
alpha = g
vect_x = vector(F,[ alpha^i  for i in range(1,n+1) ])

### Function definitions

In [2]:
def compute_lagranges(vect_x,k):
    product = 1
    for i in range(k):
        product *= X-vect_x[i]
    Lagranges = vector(Fpol,[product / (X - vect_x[i]) for i in range(k)])
    
    for i in range(k):
        denominator = F(1)
        for h in range(k):
            if h != i:
                denominator *= (vect_x[i] - vect_x[h])
        Lagranges[i] /= denominator
        
    return Lagranges

In [3]:
def generator_matrix(vect_x,k):
    Lagranges = compute_lagranges(vect_x,k)
    Gen = matrix(F,k,n)
    Gen[:,:k] = matrix.identity(k)
    for i in range(k):
        Gen[i,k:] = matrix(F,[Lagranges[i](vect_x[j]) for j in range(k,n)])
    return Gen

In [4]:
def correct_errors(r):
    R = r*Lagranges
    P = matrix(Fpol,[[G,0],[-R,1]])

    while P[1,0].degree() >= (P[1,1].degree() + k - 1):
        quo_rem = (P[0,0]).quo_rem(P[1,0])
        P = matrix(Fpol,[[0,1],[1,-quo_rem[0]]])*P
    
    row = max(P[0,0].degree(), P[0,1].degree() + k-1) > max(P[1,0].degree(), P[1,1].degree() + k-1)  
    Q01 = vector(P[row,:])
    f = (-Q01[0]).quo_rem(Q01[1])[0]
    return vector(F,[f(vect_x[i]) for i in range(n)])

## Tests

Since $g$ is a multiplicative generator of the field $\mathbb{F}_{256}$, we can see a byte $b$ (= a number between 0 and 255) as the element $g^b$ from $\mathbb{F}_{256}$

In [5]:
print(g.multiplicative_order())

255


#### Precomputation

In [53]:
k = 251
Gen = generator_matrix(vect_x,k)
Lagranges = compute_lagranges(vect_x,n)
G = 1
for i in range(n):
    G *= X-vect_x[i]

#### Encoding

In [54]:
message = vector([F.random_element() for i in range(k)])
print(message[:2])
codeword = message*Gen

(g^4 + g^2 + g + 1, g^7 + g^5 + g^4 + g + 1)


#### Adding errors

In [77]:
received = copy(codeword)
for i in range(n):
    if randint(1,1000) == 1:
        received[i] *= g^(randint(1,255))
print(received[:2])
print(received == codeword)

(g^4 + g^2 + g + 1, g^7 + g^5 + g^4 + g + 1)
False


#### Correcting errors

In [78]:
corrected = correct_errors(received)

Since the generator matrix is very systematic, we find the message by taking the $k$ first entries of the codeword

In [79]:
retrieved_message = corrected[:k]
print(retrieved_message[:2])
print(retrieved_message == message)

(g^4 + g^2 + g + 1, g^7 + g^5 + g^4 + g + 1)
True


## Simulations

In [113]:
decoding_failures = [0 for i in range(10)]
for sim_index in range(10):
    for j in range(100):
        message = vector([F.random_element() for t in range(k)])
        codeword = message*Gen

        received = copy(codeword)
        for i in range(n):
            if randint(1,1000) == 1:
                received[i] *= g^(randint(1,255))

        corrected = correct_errors(received)

        retrieved_message = corrected[:k]
        decoding_failures[sim_index] += (retrieved_message != message)
    print(sim_index)
print(decoding_failures)

0
1
2
3
4
5
6
7
8
9
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
